In [20]:
import sys
import pyspark.sql
from pyspark.sql import Row
from pyspark.sql.types import DoubleType, StringType, IntegerType, BooleanType
from pyspark.sql.functions import (
    col, lit, udf, when, expr, explode, size, substring, array, regexp_extract, concat_ws,
    collect_set, array_contains
)

spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [34]:
genetics_df = (
    spark.read.json('/Users/dsuveges/repositories/evidence_datasource_parsers/genetics_portal-2021-06-21/')
    .persist()
)

genetics_df.count()

635033

In [35]:
(
    genetics_df
    .filter(col('betaConfidenceIntervalUpper') == 'Infinity')
    .count()
)

0

In [29]:
(
    genetics_df
    .filter(
        col('oddsratioConfidenceIntervalUpper') == 'Infinity'
    )
    .select('variantId', 'diseaseFromSourceMappedId', 'studyId', 'beta', 'oddsratio', 'targetFromSourceId')
    .distinct()
#     .count()
    .show(truncate=False)
)

+-------------------+-------------------------+------------------------------------+----+---------------------+------------------+
|variantId          |diseaseFromSourceMappedId|studyId                             |beta|oddsratio            |targetFromSourceId|
+-------------------+-------------------------+------------------------------------+----+---------------------+------------------+
|17_73225039_G_GCACA|HP_0410005               |FINNGEN_R5_Q17_CHEFT_HARD_PALATE_NOS|null|1.608156093943296E234|ENSG00000141219   |
|17_73225039_G_GCACA|HP_0410005               |FINNGEN_R5_Q17_CHEFT_HARD_PALATE_NOS|null|1.608156093943296E234|ENSG00000133193   |
+-------------------+-------------------------+------------------------------------+----+---------------------+------------------+



In [36]:
(
    genetics_df
    .filter(
        (col('diseaseFromSourceMappedId') == 'HP_0410005')
        &(
            (col('targetFromSourceId') == 'ENSG00000141219')
            | (col('targetFromSourceId') == 'ENSG00000133193')
        )
    )
    .select('variantId', 'diseaseFromSourceMappedId', 'studyId', 'beta', 'oddsratio', 'targetFromSourceId')
    .distinct()
    .count()
#     .show(truncate=False)
)

2

In [33]:
df=sqlContext.createDataFrame([(None, None), (1.0, np.inf), (None, 2.0)]).toDF(["x", "y"])

replace_infs_udf = udf(
    lambda x, v: float(v) if x and np.isinf(x) else x, DoubleType()
)

df.withColumn("x1", replace_infs_udf(col("y"), lit(-99.0))).show()


NameError: name 'sqlContext' is not defined

In [22]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, StringType, IntegerType, BooleanType
from pyspark.sql.functions import (
    array, array_distinct, col, collect_set, concat, explode, lit, regexp_extract, udf,
    regexp_replace, split, trim, when, array_contains
)
import logging

spark = SparkSession.builder.appName('panelapp_parser').getOrCreate()

PMID_RE = [
    (
        r'^'                # Start of the string.
        r'[\d, ]+'          # A sequence of digits, commas and spaces.
        r'(?: |$)'          # Ending either with a space, or with the end of the string.
    ),
    (
        r'(?:PubMed|PMID)'  # PubMed or a PMID prefix.
        r'[: ]*'            # An optional separator (spaces/colons).
        r'[\d, ]'           # A sequence of digits, commas and spaces.
    )
]

def extract_pubmed_ids(publication_string):
    """Parses the publication information from the PanelApp API and extracts PubMed IDs."""
    publication_string = publication_string.strip()
    pubmed_ids = []

    for regexp in PMID_RE:  # For every known representation pattern...
        for occurrence in re.findall(regexp, publication_string):  # For every occurrence of this pattern, if any...
            pubmed_ids.extend(re.findall(r'(\d+)', occurrence))  # Extract all digit sequences (PubMed IDs).

    # Filter out '0' as a value, because it is a placeholder for a missing ID.
    return {pubmed_id for pubmed_id in pubmed_ids if pubmed_id != '0'}


def fetch_literature_references(all_panel_ids):
    """Queries the PanelApp API to extract all literature references for (panel ID, gene symbol) combinations."""
    publications = []  # Contains tuples of (panel ID, gene symbol, PubMed ID).
    for panel_id in all_panel_ids:
        logging.debug(f'Fetching literature references for panel {panel_id!r}.')
        url = f'https://panelapp.genomicsengland.co.uk/api/v1/panels/{panel_id}'
        for gene in requests.get(url).json()['genes']:
            for publication_string in gene['publications']:
                if gene['gene_data']['gene_symbol'] in gene_list:
                    print(publication_string)
                publications.extend([
                    (panel_id, gene['gene_data']['gene_symbol'], pubmed_id)
                    for pubmed_id in extract_pubmed_ids(publication_string)
                ])
    # Group by (panel ID, gene symbol) pairs and convert into a PySpark dataframe.
    return (
        spark
        .createDataFrame(publications, schema=('Panel ID', 'Symbol', 'literature'))
        .groupby(['Panel ID', 'Symbol'])
        .agg(collect_set('literature').alias('literature'))
    )

In [23]:
import re 

x = fetch_literature_references([234]).persist()
x.show()

PMID: 25998219
PMID: 24700879
PMID: 22510445
24700879
14730302.
PMID: 26572137
PMID: 24152966 - recessive mutations reported in 2 families with CAKUT, and 3 families with VACTERL.
PMID: 20301547
+--------+-------+--------------------+
|Panel ID| Symbol|          literature|
+--------+-------+--------------------+
|     234|  SLIT2|          [26026792]|
|     234|    ACE|[16116425, 30058238]|
|     234|  TBX18|          [26235987]|
|     234|  NIPBL|[16799922, 829153...|
|     234|   HAAO|[28792876, 173347...|
|     234|   ZIC3|[26294094, 20452998]|
|     234|  KMT2D|          [23535010]|
|     234|  STRA6|[26373900, 17503335]|
|     234|  ACTG2|                 [2]|
|     234| HOXA13|[24934387, 289477...|
|     234|NADSYN1|          [31883644]|
|     234|   BNC2|          [31051115]|
|     234| NOTCH2|[16773578, 221058...|
|     234| TBC1D1|                 [2]|
|     234|  WNT5A|[31032853, 27002738]|
|     234|  WBP11|          [33276377]|
|     234|  LRIG2|          [23313374]|
|    

In [13]:
x.filter(array_contains(col("literature"),'2'))

DataFrame[Panel ID: bigint, Symbol: string, literature: array<string>]

In [24]:
@udf(BooleanType())
def get_twos(array):
    if 2 in array or '2' in array:
        return True
    else:
        return False
    
x.filter(get_twos(col('literature'))).show(truncate=False)

+--------+------+-------------+
|Panel ID|Symbol|literature   |
+--------+------+-------------+
|234     |ACTG2 |[2]          |
|234     |TBC1D1|[2]          |
|234     |GRIP1 |[2, 24700879]|
|234     |SALL4 |[2]          |
|234     |TRAP1 |[2]          |
|234     |FREM1 |[2]          |
+--------+------+-------------+



In [21]:
data = requests.get('https://panelapp.genomicsengland.co.uk/api/v1/panels/234/').json()
gene_list = ['ACTG2', 'TBC1D1', 'GRIP1', 'SALL4', 'TRAP1', 'FREM1']
for gene in data['genes']:
    if gene['gene_data']['gene_symbol'] not in gene_list:
        continue
        
    print(gene['gene_data']['gene_symbol'])
    print(gene['publications'])

ACTG2
['PMID: 25998219']
FREM1
['PMID: 24700879']
GRIP1
['PMID: 22510445', '24700879', '14730302.']
TBC1D1
['PMID: 26572137']
TRAP1
['PMID: 24152966 - recessive mutations reported in 2 families with CAKUT, and 3 families with VACTERL.']
SALL4
['PMID: 20301547']
